!git clone https://github.com/Graylab/MaskedProteinEnT.git
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements_torch191.txt

In [ ]:
mkdir -p training_datasets

wget -P training_datasets https://zenodo.org/records/13831403/files/ids_train_casp12nr50_nr70Ig_nr40Others.fasta

wget -P training_datasets https://zenodo.org/records/13831403/files/sidechainnet_casp12_50.pkl

wget -P training_datasets https://zenodo.org/records/13831403/files/AbSCSAbDAb_trainnr90_bkandcbcoords_aug2022.h5
wget -P training_datasets https://zenodo.org/records/13831403/files/ppi_trainset_5032_noabag_aug2022.h5
wget -P training_datasets https://zenodo.org/records/13831403/files/train_af_paired_nr70.h5

# Is this the test set?
wget -P training_datasets https://zenodo.org/records/13831403/files/testset_rabd-dms-vhh_backboneandcb_oct2022.h5

In [ ]:
### MODEL AND TRAINING PARAMETERS
LAYERS=4
HEADS=8
DIM=256
OLD=0
BS=1
SS=50 # protein model was trained on 90ss
save_every=5
gmodel=egnn-trans-ma
atom_types=backbone_and_cb
NN=48

### Training seed and output directory
SEED=1
MODELS_DIR=models_out_dir_seed_$SEED
EPOCHS=10

### WANDB ENTITY
WANDB_ENTITY="fadh-johns-hopkins-university"
